# WATER CHEMICAL POTENTIAL
### MSI project 2

#### Andreu Bofill Pumarola
____________


#### INTRODUCTION
The aim of this project is to create a python program which, given a set of molecular dynamics trajectories, computes the volumetric map of the chemical potential for water. Furthermore, the output of this program is a .cube file that you can open with VMD and observe the isosurface of water distribution. 

First of all we need to import the programs and functions we will use

In [1]:
from htmd import *
import numpy as np
import os
from htmd.molecule.util import maxDistance
import math
from htmd.molecule.util import writeVoxels

New HTMD version (1.0.22) is available. You are currently on (1.0.16). Use 'conda update htmd' to update to the new version.



With the simlist function we import all the files needed. We use a pdb structure with 10 diferent simulations. So, we  have 10 simulations with 2000 trajectories/frames.

In [2]:
sims = simlist(glob('pub.htmd.org/CXCL12-confAnalysis/*/'), glob('pub.htmd.org/CXCL12-confAnalysis/*/structure.pdb'), glob('pub.htmd.org/CXCL12-confAnalysis/*/'))

Creating simlist: 100% (10/10) [###################################] eta 00:01 -


Then, we need to charge the structure as a Molecule and read the trajectories of each simulation.
The htmd function `align`, helps us to fix the molecule. and  it and save each one of them in a list than will make it easy to iterate throw. 
On this iteration we create another list of maximum coordenates, that subsequently we will use to extract the farthest x-coordinate of all the 10 pdb simuations. 

In [3]:
mollist=[]
maxcoord = []
i = 0
for el in sims:
        molec = Molecule(el.molfile)
        molec.read(el.trajectory)
        molec.wrap()
        molec.align("protein", refmol=Molecule(el.molfile))
        mollist.append(molec)
        maxcoord.append(np.amax(molec.coords[:,0,0]))
        maxcoord

In [4]:
mxc = int(max(maxcoord))
mxc_alt = mxc * 1.1

Finishing the input reading and  loading, we need to construct the 3-dimensional regular grid, where each dimension of it will have a size of 1Å. This matrix will give us a complete water distribution over all the 3-Dimensional space of the molecule. To construct it, we use the numpy function `zeros` . THe dimensions of the grid will be given by the maximum coordinate extracted previously and multiplied with an arbitrary number to assure a more comfortable grid to work. 

In [5]:
box=np.zeros([int(mxc*2.3),int(mxc*2.3),int(mxc*2.3)])

Next, we need to iterate over each Molecule. And within each iteration, we need to move the Molecule to avoid having any negative coordinates that will dificult our work. We also need to filter the Molecule to obtain only the Oxigen atoms of the water molecules. 


Then we iterate over each frame of the Molecule and also over each one of the Oxigen atom coordinates (x,y,z) within this frame. With this coordinates, we search the cell in the grid that this atom is located on and sum 1 into the count inside this cell. This approach will give us a oxigen atom counting 3D matrix.

In [6]:
for mol in mollist:
    mol.moveBy([mxc_alt,mxc_alt,mxc_alt])
    mol.filter("water and name OH2")
    coords=mol.coords
    for i in range(coords.shape[2]):
        for co in coords[:,:,i]:
            try:
                a = int(np.floor(co[0]))
                b = int(np.floor(co[1]))
                c = int(np.floor(co[2]))
                box[a,b,c] += 1
            except:
                pass

Once we finish to fill the matrix with all the oxigen atom coordinates (x,y,z), we add a really low value (like `1e-41`) to avoid an error with all the 0's cells when we apply the `log(p(x,y,z))`.
The boxsum represent the sum of all atoms computed inside the grid.

In [7]:
box += 1e-41
boxsum = box.sum()
boxsum

141319993.0

To compute the chemical potential (`G(x,y,z)= - kB T log(p(x,y,z))`), we need to know the water occupancy, which is represented by the `log` variable as the oxigen distribution grid divided by the total number of atoms inside the grid. k is the Boltzmann constant (kB) and T is the temperature (298K)

In [8]:
log = np.log(box/(boxsum))
k = 0.001987191
G= -k*298*log

Finally, we need to save the volumetric file, that will allows us to visualize the isosurface of water distribution.

In [9]:
writeVoxels(G,"isos_cube.cube",np.array([0,0,0]),[mxc*2.3,mxc*2.3,mxc*2.3],np.array([1,1,1])) 

To visualize the isosurface of water distribution, you can open the VMD with the `isos_cube.cube` file and then change the Drawing method on Representations from lines to isosurface and increase a bit the isovalue range.

#### OPTIONAL SECTION: GAUSSIAN NORMALITZATION

<p> At this section we will use now a Gaussian filter to distribute the occupancy of a water oxygen over neighboring grib points, having the effect of interpolating between points.</p>
<p>This normalitzation is used before the chemical potential calculation. So, after computing the gaussian_filter function, we need to load compute again te chemical potential formula, but using now the Gaussian normalitzation grid. Finally, we use the already seened `writeVoxels` function to visualize the volumetric file. </p>

In [10]:
from scipy.ndimage.filters import gaussian_filter

In [11]:
sigma = 1.5
Gauss_box = gaussian_filter(box, sigma)

In [12]:
log = np.log(Gauss_box/(boxsum))
k = 0.001987191
G= -k*298*log

In [13]:
writeVoxels(G,"cubefile.cube",np.array([0,0,0]),[mxc*2.3,mxc*2.3,mxc*2.3],np.array([1,1,1])) 